# Classification supervisée des données numériques (quantitatives)
Dans ce lab, nous allons voir comment réaliser la tàche de la classification supervisée en utilisant Python + Jupyter Notebook avec les librairies sklearn et keras.


## 1. importation des librairies
Avec Pandas on peut manipuler lire et écrire nos jeux de données, généralement avec une extension .csv



In [1]:
import pandas as pd   # lire et manipuler les IO .csv
#import numpy as np   
#import matplotlib 
#import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.model_selection import train_test_split #découper l'ensemble de train et de test de maniere aléatoire
#import math

# 2. Importation des données
Avec la fonction read_csv de Pandas: on peut mettre dans notre dataframe le contenu du fichier csv, en indiquant comme paramètre (1: le chemin ou la source où se trouve le fichier csv, 2: les séparateurs entre les valeurs dans notre cas ces des vergules) en troisième position, un paramètre facultatif pour spécifier le type d'encodage de notre fichier exemple encoding ="UTF8".

In [2]:
df=pd.read_csv('/home/notebooks/Amine/Bank DataSets Experimentations/Give me some credit.csv', sep=',')#lire le fichier csv 
df.head()  # afficher les 5 premieres lignes

,ID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


## 3. Statistiques descriptives élémentaires

Lire les informations sur nos données (Types d'attributs, valeurs manquantes...)
Pandas nous permet de voir les informations sur notre benchmark
exemple: avec dataframe.info() il nous affiche tout les attributs de notre fichier avec le type de donnée et le nombre de valeurs de chaque colonne

dataframe.columns permet de citer les noms de toutes les colonnes

In [3]:
df.info() #donner les infos de notre data frame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
ID                                      150000 non-null int64
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
Age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(8)
memory usage: 13.7 MB


In [4]:
df.columns # citer les colonnes

Index(['ID', 'SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'Age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')

## 4. préparation des données
Dans cette étape nous déterminons les attributs choisis pour l'entrainement et nous définissons l'attribut "classe" de notre benchmark

In [5]:
# définir les attraibuts qui nous intéréssent, ici j'ai éliminé les attributs qui contiennent des nulles
df_features = df[['ID', 'RevolvingUtilizationOfUnsecuredLines', 'Age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', #'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       #'NumberOfDependents'
                 ]]

In [6]:
# définir l'attribut classe
df_labels = df[['SeriousDlqin2yrs']]

Si on veut schématiser la distribution de nos classes, il suffit de faire appel à la libraire seaborn, en suite définir l'attribut concerné 

In [7]:
import seaborn as sns
# schématiser la distribution des classes
sns.countplot(df['SeriousDlqin2yrs'])

## 5. Diviser le dataset en données d'entrainement et données de teste
Ceci est réalisable avec sklearn qui permet de prendre aléatoirement des données de teste à partir du benchmark et laisser le reste pour l'apprentissage.
La fonction train_test_split(param1,param2,param3,param4) prends 4 paramétres:
le premier dédié à l'ensemble d'entrainement, le deuxième à l'ensemble de teste, le troisième c'est le paramètre du % de l'ensemble de test (généralement entre 15 et 40%), 

le 4 ème paramétre (facultatif) pour spécifier quel type de fonction random utiliser:
si vous utilisez random_state = some_number, vous pouvez garantir que la sortie de Run 1 sera égale à la sortie de Run 2, c'est-à-dire que votre split sera toujours le même. Peu importe ce que le nombre réel random_state est 42, 0, 21, ... L'important est que chaque fois que vous utilisez 42, vous obtiendrez toujours la même sortie la première fois que vous faites la division. Ceci est utile si vous voulez des résultats reproductibles, par exemple dans la documentation, afin que tout le monde puisse toujours voir les mêmes nombres lors de l'exécution des exemples.

Cette fonction retourne 4 sorties: 
La 1ere est le sous-ensembles aléatoires d'entrainement 
La 2éme est le vecteur de leurs labels (leurs classes).
La 3ème est le sous-ensemble aléatoire pour le teste.
La 4ème est le vecteur de leurs labels (leurs classes).


In [8]:
from sklearn.model_selection import train_test_split
#decouper le data set en 30% pour test et 70% pour train
X_train, X_test, y_train, y_test = train_test_split(df_features, df_labels, test_size=0.3, random_state=42)

.shape permet de savoir la dimension d'un ensemble.
Par exemple ici l'ensemble d'entrainement est composé de 105000 lignes et 9 colonnes. 

In [9]:
print('x_train shape:', X_train.shape) # .shape permet de voir la
print('x_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (105000, 9)
x_test shape: (45000, 9)
y_train shape: (105000, 1)
y_test shape: (45000, 1)


## 6. Appliquer les Algorithmes de la fouille de données 
### sklearn : 
Contient la plus part des algorithmes de data mining , ici par exemple on fait appel aux algorithmes Naive Bayes , Support Vector Machines, et au KNN (algorithme des k plus proches voisins) 


In [10]:
########## DATA Mining Approaches

In [11]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score #confusion_matrix
#import matplotlib.pyplot as plt 
#import itertools    #pour dessiner la matrice de confusion

In [12]:
#1 Bernouli Naive Bayes
import time
bnb= BernoulliNB()
tps1 = time.clock()
#fitting
bnb.fit(X_train, y_train) #.values.ravel()
# y = ravel(a) : If a is a matrix, y is a 1-D ndarray, otherwise y is an array of the same subtype as a. 
#The shape of the returned array is (a.size,). 
#Matrices are special cased for backward compatibility.

ypredBnb = bnb.predict(X_test)
tps2 = time.clock() 
ts = tps2 - tps1
print("time = ",ts)
print ('Bernoulli accuracy score')
print (accuracy_score(y_test, ypredBnb))

time =  0.04556199999999988
Bernoulli accuracy score
0.9266666666666666


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
#2 Multinomial Naive Bayes
mnb=MultinomialNB()
#fitting
tpp1= time.clock()
mnb.fit(X_train, y_train.values.ravel())
ypredMnb = mnb.predict(X_test)
tpp2= time.clock()
tp=tpp2-tpp1
print('time =' ,tp)
print ('Multinomial accuracy score')
print (accuracy_score(y_test, ypredMnb))

time = 0.021079000000000292
Multinomial accuracy score
0.26982222222222224


In [17]:
#3 SVMM
from sklearn.svm import LinearSVC, NuSVC, SVC

In [18]:
lsvm= LinearSVC()

tp1=time.clock()
#fitting
lsvm.fit(X_train, y_train.values.ravel())
#ypred
ypredLsvm = lsvm.predict(X_test)
tp2=time.clock()
tp=tp2-tp1
print("time =" , tp)
print ('Linear Svm accuracy score')
print (accuracy_score(y_test, ypredLsvm))

time = 8.861974
Linear Svm accuracy score
0.9211555555555555


In [19]:
# 4 KNN Uniform
from sklearn.neighbors import KNeighborsClassifier

In [20]:
#UNIFORM KNN
knn_uni = KNeighborsClassifier(n_neighbors=13, weights = 'uniform')
#fitting
tt1=time.clock()
knn_uni.fit(X_train, y_train.values.ravel())
#ypred
ypred_knn_uni = knn_uni.predict(X_test)
tt2=time.clock()
tt=tt2 - tt1
print("time =",tt )
print ('KNN Uniform accuracy score')
print (accuracy_score(y_test, ypred_knn_uni))

time = 1.1546389999999995
KNN Uniform accuracy score
0.9337777777777778


In [21]:
# 5 KNN Distant
knn_dist = KNeighborsClassifier(n_neighbors=13, weights = 'distance')
#fitting
tt1 = time.clock()
knn_dist.fit(X_train, y_train.values.ravel())
#ypred
ypred_knn_dist = knn_dist.predict(X_test)
tt2=time.clock()
tt=tt2 - tt1
print("time =",tt )
print ('Distant accuracy score')
print (accuracy_score(y_test, ypred_knn_dist))

time = 1.1826849999999993
Distant accuracy score
0.9335777777777777


## 7. Appliquer les réseaux de neuronnes pour la classification des données
En utilisant KERAS on peut définir notre modèle : en spécifiant le nombre de couches,le nombre de neuronnes dans chaque couche, et la fonction d'activation.

In [22]:
############# Machine Learning approaches

In [23]:
batch_size = 32#32
epochs = 15
from keras.callbacks import ModelCheckpoint

checkpoint = [ModelCheckpoint(filepath='modNN_.hdf5')]

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to preventoverfitting
import keras
from keras.models import Sequential

#import sys
#import numpy as np
import tensorflow as tf
#from datetime import datetime
from keras.layers import Dense, Activation, Flatten, Dropout

#device_name="/gpu:0"

#device_name="/device:CPU:0"   # UTILISATION DU GPU #4


#with tf.device(device_name):
model = Sequential()
model.add(Dense(514, input_shape=(9,)))#max_words, X_train.shape[1:]
model.add(Activation('relu'))
model.add(Dense(256 ))
model.add(Activation('relu'))
model.add(Dense(128 ))
model.add(Activation('relu'))
model.add(Dropout(0.5))
    #model.add(Dense(num_classes,))#num_classes
    #model.add(Activation('softmax'))
model.add(Dense(1, activation='sigmoid'))
    
model.compile(loss='binary_crossentropy',#sparse_categorical_crossentropy
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=checkpoint, # Checkpoint
                    verbose=1,
                    validation_split=0.1)

Train on 94500 samples, validate on 10500 samples
Epoch 1/15
94500/94500 [==============================] - 30s 312us/step - loss: 1.0939 - acc: 0.9321 - val_loss: 1.0546 - val_acc: 0.9346
Epoch 2/15
94500/94500 [==============================] - 12s 122us/step - loss: 1.0846 - acc: 0.9327 - val_loss: 1.0546 - val_acc: 0.9346
Epoch 3/15
94500/94500 [==============================] - 11s 121us/step - loss: 1.0846 - acc: 0.9327 - val_loss: 1.0546 - val_acc: 0.9346
Epoch 4/15
94500/94500 [==============================] - 12s 122us/step - loss: 1.0846 - acc: 0.9327 - val_loss: 1.0546 - val_acc: 0.9346
Epoch 5/15
94500/94500 [==============================] - 12s 122us/step - loss: 1.0846 - acc: 0.9327 - val_loss: 1.0546 - val_acc: 0.9346
Epoch 6/15
94500/94500 [==============================] - 12s 123us/step - loss: 1.0846 - acc: 0.9327 - val_loss: 1.0546 - val_acc: 0.9346
Epoch 7/15
94500/94500 [==============================] - 12s 123us/step - loss: 1.0846 - acc: 0.9327 - val_loss: 1.

In [25]:
score = model.evaluate(X_test, y_test,
                       batch_size=32, verbose=1)

print('Test accuracy:', score[1])

45000/45000 [==============================] - 2s 40us/step
Test accuracy: 0.9337777777777778


In [ ]:
#CNN model 3

In [30]:
from keras.layers import Conv1D, MaxPooling1D,Convolution2D,Conv2D

In [31]:
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
#def create_model():
# ValueError: Error when checking model input: expected embedding_2_input to have shape (None, 300) but got array with shape (33908, 42)
model = Sequential()
model.add(Embedding(2, 2, input_length=9))
model.add(Dropout(0.2))
model.add(Conv1D(2, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(data, np.array(balanced_labels), validation_split=0.5, epochs=3)


#model.compile(loss='binary_crossentropy',#sparse_categorical_crossentropy
 #             optimizer='adam',
  #            metrics=['accuracy'])
model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=checkpoint, # Checkpoint
                    verbose=1,
                    validation_split=0.1)
#    return model
#CNN(5,100)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 94500 samples, validate on 10500 samples
Epoch 1/15
94500/94500 [==============================] - 25s 262us/step - loss: 0.2437 - acc: 0.9335 - val_loss: 0.2466 - val_acc: 0.9303
Epoch 2/15
94500/94500 [==============================] - 22s 235us/step - loss: 0.2348 - acc: 0.9335 - val_loss: 0.2426 - val_acc: 0.9303
Epoch 3/15
94500/94500 [==============================] - 22s 233us/step - loss: 0.2333 - acc: 0.9336 - val_loss: 0.2464 - val_acc: 0.9303
Epoch 4/15
94500/94500 [==============================] - 22s 232us/step - loss: 0.2319 - acc: 0.9337 - val_loss: 0.2492 - val_acc: 0.9304
Epoch 5/15
94500/94500 [==============================] - 22s 231us/step - loss: 0.2319 - acc: 0.9337 - val_loss: 0.2599 - val_acc: 0.9303
Epoch 6/15
94500/94500 [==============================] - 21s 227us/step - loss: 0.2313 - acc: 0.9338 - val_loss: 0.2443 - val_acc: 0.9303
Epoch 7/15
94500/94500 [=========

In [33]:
score = model.evaluate(X_test, y_test,
                       batch_size=32, verbose=1)
#print('Test score:', score[0])
print('Test accuracy:', score[1])

45000/45000 [==============================] - 3s 63us/step
Test accuracy: 0.9340222222222222
